&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE_RIVA_BOOTCAMP.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;
[1](question-answering-training.ipynb)
[2]
[3](question-answering-Exercise.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](question-answering-Exercise.ipynb)

# Deploying a Question Answering Model in Riva

[Transfer Learning Toolkit](https://developer.nvidia.com/transfer-learning-toolkit) (TLT) provides the capability to export your model in a format that can deployed using [NVIDIA Riva](https://developer.nvidia.com/riva), a highly performant application framework for multi-modal conversational AI services using GPUs. 

This tutorial explores taking an .riva model, the result of `tlt question_answering export` command, and leveraging the Riva ServiceMaker framework to aggregate all the necessary artifacts for Riva deployment to a target environment. Once the model is deployed in Riva, you can issue inference requests to the server. We will demonstrate how quick and straightforward this whole process is. 

---
## Learning Objectives
In this notebook, you will learn how to:  
- Use Riva ServiceMaker to take a TLT exported .riva and generate a model [resources](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart)
- Deploy the model(s) locally  on the Riva Server
- Send inference requests from a demo client using Riva API bindings.

---
## Pre-requisites

To follow along, please make sure:
- You have access to NVIDIA NGC, and are able to download the Riva Quickstart [resources](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart)
- Have an .riva model file that you wish to deploy. You can obtain this from `tlt <task> export` (with `export_format=RIVA`). Please refer the tutorial on *Question Answering using Transfer Learning Toolkit* for more details on training and exporting an .riva model.
- You have followed the steps in the setup notebook and have a running instance of the RIVA server ready

### Riva-deploy

The deployment tool takes as input one or more Riva Model Intermediate Representation (RMIR) files and a target model repository directory. It creates an ensemble configuration specifying the pipeline for the execution and finally writes all those assets to the output model repository directory.

In [ ]:
# Syntax: riva-deploy -f dir-for-rmir/model.rmir:key output-dir-for-repository
!docker run --rm --gpus 1 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
            riva-deploy -f /data/question-answering.rmir:$KEY /data/models

Remember to modify config.sh to enable relevant Riva services (nlp for Question Answering model), provide the encryption key, and path to the model repository (`riva_model_loc`) generated in the previous step among other configurations. 


---
## Run Inference
Once the Riva server is up and running with your models, you can send inference requests querying the server. 

To send GRPC requests, you can install Riva Python API bindings for client. This is available as a pip .whl with the QuickStart.


In [ ]:
# IMPORTANT: Set the name of the whl file
RIVA_API_WHL = "<add riva api .whl file name>"

In [ ]:
# Install client API bindings
!cd $RIVA_DIR && pip install $RIVA_API_WHL

The following code sample shows how you can perform inference using Riva Python API gRPC bindings:

In [ ]:
import grpc
import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv


grpc_server =  "localhost:50051"
channel = grpc.insecure_channel(grpc_server)
riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(channel)

req = rnlp.NaturalQueryRequest()

test_context = "In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the 2005 drought."\
                "The affected region was approximate 1,160,000 square miles (3,000,000 km2) of rainforest, compared to 734,000 square miles (1,900,000 km2)" \
                " in 2005. The 2010 drought had three epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern part." \
                " The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 gigatons of carbon dioxide; during 2005" \
                "instead 5 gigatons were released and in 2010 8 gigatons were released."
req.query = "How many tons of carbon are absorbed the Amazon in a typical year?"
req.context = test_context
resp = riva_nlp.NaturalQuery(req)
print(resp)

`NOTE`: You could also run the above inference code from inside the Riva Client container. The QuickStart provides a script `riva_start_client.sh` to run the container. It has more examples for different services.

 You can stop all docker container before shutting down the jupyter kernel. **Caution: The following command will stop all running containers**

In [ ]:
!docker stop $(docker ps -a -q)

---
## What's next?
You could train your own custom models in TLT and deploy them in Riva! You could scale up your deployment using Kubernetes with the Riva AI Services Helm Chart, which will pull the relevant Images and download model artifacts from NGC, generate the model repository, start and expose the Riva speech services.